### Sample SQL Query to get golfer handicapping data

Save query results to CSV file: `./handicap_data/handicap_data_golfer_###.csv`

```sql
SELECT golfer.id as golfer_id, golfer.name as golfer_name,
	roundgolferlink.playing_handicap, course.name as course_name,
	track.name as track_name, tee.name as tee_name,	tee.rating,	tee.slope,
	round.id as round_id, round.date_played, holeresult.id as holeresult_id,
	hole.number, hole.par, hole.yardage, holeresult.handicap_strokes,
	holeresult.gross_score,	holeresult.adjusted_gross_score, holeresult.net_score
FROM public.round
INNER JOIN public.roundgolferlink ON roundgolferlink.round_id = round.id
INNER JOIN public.golfer ON golfer.id = roundgolferlink.golfer_id
INNER JOIN public.holeresult ON holeresult.round_id = round.id
INNER JOIN public.hole ON hole.id = holeresult.hole_id
INNER JOIN public.tee ON tee.id = round.tee_id
INNER JOIN public.track ON track.id = tee.track_id
INNER JOIN public.course ON course.id = track.course_id
WHERE public.golfer.name LIKE 'Andris Jaunzemis'
ORDER BY round.date_played, hole.number;
```

In [27]:
# LOAD HANDICAP DATA FROM FILE

from dataclasses import dataclass
from datetime import datetime

import pandas as pd
import seaborn as sns

@dataclass
class HoleHandicapData:
    holeresult_id: int
    number: int
    par: int
    yardage: int
    handicap_strokes: int
    gross_score: int
    adjusted_gross_score: int
    net_score: int

@dataclass
class RoundHandicapData:
    round_id: int
    golfer_id: int
    # tee_id: int # TODO: include this too?
    date_played: datetime
    course_name: str
    track_name: str
    tee_name: str
    rating: float
    slope: float
    holes: list[HoleHandicapData]

    @property
    def par(self) -> int:
        return sum(h.par for h in self.holes)

    @property
    def gross_score(self) -> int:
        return sum(h.gross_score for h in self.holes)
    
    @property
    def gross_score_to_par(self) -> int:
        return self.gross_score - self.par

    @property
    def adjusted_gross_score(self) -> int:
        return sum(h.adjusted_gross_score for h in self.holes)
    
    @property
    def net_score(self) -> int:
        return sum(h.net_score for h in self.holes)

    @property
    def scoring_differential(self) -> float:
        return (113 / self.slope) * (self.adjusted_gross_score - self.rating)


DATA_FILE = "./handicap_data/handicap_data_golfer_003.csv"
df = pd.read_csv(DATA_FILE)

for round_id, round_df in df.groupby("round_id"):
    round_holes: list[HoleHandicapData] = []
    for idx, round_row in round_df.iterrows():
        round_holes.append(HoleHandicapData(
            holeresult_id=round_row["holeresult_id"],
            number=round_row["number"],
            par=round_row["par"],
            yardage=round_row["yardage"],
            handicap_strokes=round_row["handicap_strokes"],
            gross_score=round_row["gross_score"],
            adjusted_gross_score=round_row["adjusted_gross_score"],
            net_score=round_row["net_score"],
        ))
    
    round_data = RoundHandicapData(
        round_id=round_df.iloc[0]["round_id"],
        golfer_id=round_df.iloc[0]["golfer_id"],
        # tee_id=round_df.iloc[0]["tee_id"], # TODO: include this too?
        date_played=pd.to_datetime(round_df.iloc[0]["date_played"]),
        course_name=round_df.iloc[0]["course_name"],
        track_name=round_df.iloc[0]["track_name"],
        tee_name=round_df.iloc[0]["tee_name"],
        rating=round_df.iloc[0]["rating"],
        slope=round_df.iloc[0]["slope"],
        holes=round_holes,
    )
    print(f"{round_data.date_played.strftime("%Y-%m-%d")} | {round_data.gross_score} ({round_data.gross_score_to_par:+3d}) | {round_data.scoring_differential:5.2f} | {round_data.course_name}, {round_data.track_name}, {round_data.tee_name}")


FileNotFoundError: [Errno 2] No such file or directory: './data/handicap_data_golfer_003.csv'